In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
%matplotlib inline
import pathlib

In [2]:
batch_size=256
image_size=(300, 300)

In [3]:
dir_path = pathlib.Path.cwd()
dir_path = str(dir_path)
dir_path

'H:\\IntensiveGitHub\\Intensive'

In [4]:
train_dataset = image_dataset_from_directory('All',
                                             subset='training',
                                             seed=42,
                                             label_mode='categorical',
                                             validation_split=0.2,
                                             batch_size=batch_size,
                                             image_size=image_size)

Found 1185 files belonging to 25 classes.
Using 948 files for training.


In [5]:
# generate
validation_dataset = image_dataset_from_directory('All',
                                                  subset='validation',
                                                  label_mode='categorical',
                                                  seed=42,
                                                  validation_split=0.2,
                                                  batch_size=batch_size,
                                                  image_size=image_size)

Found 1185 files belonging to 25 classes.
Using 237 files for validation.


In [6]:
class_names = train_dataset.class_names
class_names

['Baked_cheese_omelet',
 'Bread_made_from_wheat_flour',
 'Butter',
 'Casserole',
 'Cheese',
 'Cheese_casserole',
 'Cheese_sandwich',
 'Cherry_Sauce',
 'Coffee',
 'Compote',
 'Cookie',
 'Fresh_seasonal_fruits',
 'Juice',
 'Kissel',
 'Milk',
 'Mix',
 'Pasta_with_cheese',
 'Pie_with_raisins',
 'Porridge_with_milk',
 'Rice porridge',
 'Tea',
 'TeaWithMilk',
 'Water',
 'buckwheat porridge',
 'millet porridge']

In [7]:
# test_dataset = image_dataset_from_directory('Training',
#                                             batch_size=batch_size,
#                                             image_size=image_size)

In [8]:
# test_dataset.class_names

In [9]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
# test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [10]:
from keras.layers import BatchNormalization

# # Создаем последовательную модель
# model = Sequential()
# # Сверточный слой
# model.add(Conv2D(16, (5, 5), padding='same',
#                  input_shape=(300, 300, 3), activation='relu'))
# # Слой подвыборки
# model.add(MaxPooling2D(pool_size=(2, 2)))
# # Сверточный слой
# model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))
# # Слой подвыборки
# model.add(MaxPooling2D(pool_size=(2, 2)))
# # Сверточный слой
# model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
# # Слой подвыборки
# model.add(MaxPooling2D(pool_size=(2, 2)))
# # Сверточный слой
# model.add(Conv2D(128, (5, 5), activation='relu', padding='same'))
# # Слой подвыборки
# model.add(MaxPooling2D(pool_size=(2, 2)))
# # Полносвязная часть нейронной сети для классификации
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.2))
# # Выходной слой, 131 нейрон по количеству классов
# model.add(Dense(25, activation='softmax'))
# Модель
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(25, activation='softmax'))

# компиляция модели многолассовой классификации
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 298, 298, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 149, 149, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 64)      18496     
                                                                 
 batch_normalization_1 (Batc  (None, 147, 147, 64)     2

In [11]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(patience=10)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', patience = 3, factor = 0.5, min_lr = 0.00001)
callbacks = [early_stopping, reduce_lr]

In [12]:
with tf.compat.v1.Session() as sess:
    devices = sess.list_devices()

In [ ]:
with tf.device('/gpu:0'):
    history = model.fit(
        train_dataset,
        epochs = 10,
        validation_data = validation_dataset,
        callbacks = callbacks
    )

Epoch 1/10


In [ ]:
# Оцениваем качество обучения модели на тестовых данных
scores = model.evaluate(test_dataset, verbose=1)

In [ ]:
print("Доля верных ответов на тестовых данных, в процентах:", round(scores[1] * 100, 4))

In [ ]:
plt.plot(history.history['accuracy'],
         label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_accuracy'],
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'],
         label='Ошибка на обучающем наборе')
plt.plot(history.history['val_loss'],
         label='Ошибка на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Ошибка')
plt.legend()
plt.show()

In [ ]:
model.save("fruits_360_model.h5")

In [ ]:
files.download("fruits_360_model.h5")